In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from post_clustering import spectral_clustering, acc, nmi
import scipy.io as sio
import math
from torch.autograd import Variable
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
from sklearn import cluster
from scipy.linalg import sqrtm
from numpy.linalg import inv

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
data = sio.loadmat('datasets/ORL_32x32.mat')
x, y = data['fea'].reshape((-1, 1, 32, 32)), data['gnd']
y = np.squeeze(y - 1)  

# network and optimization parameters
num_sample = x.shape[0]
num_classes = 40 
num_fea = 80 
channels = [1, 3, 3, 5]
kernels = [3, 3, 3]
gamma1, gamma2, gamma3, gamma4 = 0.1 , 0.01 , 8 , 1.2
ratio = 0.8      
t0 = 5 
t_dp = 12
k_dp = 10
shrink = 1
epochs = 940
lr = 1e-3
update = 1 
p = 1 
rw_time = 2000

# post clustering parameters
alpha = 0.2  # threshold of C
dim_subspace = 4  # dimension of each subspace
ro = 1  #

In [4]:
class Conv2dSamePad(nn.Module):
    """
    Implement Tensorflow's 'SAME' padding mode in Conv2d.
    When an odd number, say `m`, of pixels are need to pad, Tensorflow will pad one more column at right or one more
    row at bottom. But Pytorch will pad `m+1` pixels, i.e., Pytorch always pads in both sides.
    So we can pad the tensor in the way of Tensorflow before call the Conv2d module.
    """

    def __init__(self, kernel_size, stride):
        super(Conv2dSamePad, self).__init__()
        self.kernel_size = kernel_size if type(kernel_size) in [list, tuple] else [kernel_size, kernel_size]
        self.stride = stride if type(stride) in [list, tuple] else [stride, stride]

    def forward(self, x):
        in_height = x.size(2)
        in_width = x.size(3)
        out_height = math.ceil(float(in_height) / float(self.stride[0]))
        out_width = math.ceil(float(in_width) / float(self.stride[1]))
        pad_along_height = ((out_height - 1) * self.stride[0] + self.kernel_size[0] - in_height)
        pad_along_width = ((out_width - 1) * self.stride[1] + self.kernel_size[1] - in_width)
        pad_top = math.floor(pad_along_height / 2)
        pad_left = math.floor(pad_along_width / 2)
        pad_bottom = pad_along_height - pad_top
        pad_right = pad_along_width - pad_left
        return F.pad(x, [pad_left, pad_right, pad_top, pad_bottom], 'constant', 0)


In [5]:
class ConvTranspose2dSamePad(nn.Module):
    """
    This module implements the "SAME" padding mode for ConvTranspose2d as in Tensorflow.
    A tensor with width w_in, feed it to ConvTranspose2d(ci, co, kernel, stride), the width of output tensor T_nopad:
        w_nopad = (w_in - 1) * stride + kernel
    If we use padding, i.e., ConvTranspose2d(ci, co, kernel, stride, padding, output_padding), the width of T_pad:
        w_pad = (w_in - 1) * stride + kernel - (2*padding - output_padding) = w_nopad - (2*padding - output_padding)
    Yes, in ConvTranspose2d, more padding, the resulting tensor is smaller, i.e., the padding is actually deleting row/col.
    If `pad`=(2*padding - output_padding) is odd, Pytorch deletes more columns in the left, i.e., the first ceil(pad/2) and
    last `pad - ceil(pad/2)` columns of T_nopad are deleted to get T_pad.
    In contrast, Tensorflow deletes more columns in the right, i.e., the first floor(pad/2) and last `pad - floor(pad/2)`
    columns are deleted.
    For the height, Pytorch deletes more rows at top, while Tensorflow at bottom.
    In practice, we usually want `w_pad = w_in * stride`, i.e., the "SAME" padding mode in Tensorflow,
    so the number of columns to delete:
        pad = 2*padding - output_padding = kernel - stride
    We can solve the above equation and get:
        padding = ceil((kernel - stride)/2), and
        output_padding = 2*padding - (kernel - stride) which is either 1 or 0.
    But to get the same result with Tensorflow, we should delete values by ourselves instead of using padding and
    output_padding in ConvTranspose2d.
    To get there, we check the following conditions:
    If pad = kernel - stride is even, we can directly set padding=pad/2 and output_padding=0 in ConvTranspose2d.
    If pad = kernel - stride is odd, we can use ConvTranspose2d to get T_nopad, and then delete `pad` rows/columns by
    ourselves; or we can use ConvTranspose2d to delete `pad - 1` by setting `padding=(pad - 1) / 2` and `ouput_padding=0`
    and then delete the last row/column of the resulting tensor by ourselves.
    Here we implement the former case.
    This module should be called after the ConvTranspose2d module with shared kernel_size and stride values.
    And this module can only output a tensor with shape `stride * size_input`.
    A more flexible module can be found in `yaleb.py` which can output arbitrary size as specified.
    """

    def __init__(self, kernel_size, stride):
        super(ConvTranspose2dSamePad, self).__init__()
        self.kernel_size = kernel_size if type(kernel_size) in [list, tuple] else [kernel_size, kernel_size]
        self.stride = stride if type(stride) in [list, tuple] else [stride, stride]

    def forward(self, x):
        in_height = x.size(2)
        in_width = x.size(3)
        pad_height = self.kernel_size[0] - self.stride[0]
        pad_width = self.kernel_size[1] - self.stride[1]
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left
        return x[:, :, pad_top:in_height - pad_bottom, pad_left: in_width - pad_right]

In [6]:
class ConvAE(nn.Module):
    def __init__(self, channels, kernels):
        """
        :param channels: a list containing all channels including the input image channel (1 for gray, 3 for RGB)
        :param kernels:  a list containing all kernel sizes, it should satisfy: len(kernels) = len(channels) - 1.
        """
        super(ConvAE, self).__init__()
        assert isinstance(channels, list) and isinstance(kernels, list)
        self.encoder = nn.Sequential()
        for i in range(1, len(channels)):
            #  Each layer will divide the size of feature map by 2
            self.encoder.add_module('pad%d' % i, Conv2dSamePad(kernels[i - 1], 2))
            self.encoder.add_module('conv%d' % i,
                                    nn.Conv2d(channels[i - 1], channels[i], kernel_size=kernels[i - 1], stride=2))
            self.encoder.add_module('relu%d' % i, nn.ReLU(True))

        self.decoder = nn.Sequential()
        channels = list(reversed(channels))
        kernels = list(reversed(kernels))
        for i in range(len(channels) - 1):
            # Each layer will double the size of feature map
            self.decoder.add_module('deconv%d' % (i + 1),
                                    nn.ConvTranspose2d(channels[i], channels[i + 1], kernel_size=kernels[i], stride=2))
            self.decoder.add_module('padd%d' % i, ConvTranspose2dSamePad(kernels[i], 2))
            self.decoder.add_module('relud%d' % i, nn.ReLU(True))

    def forward(self, x):
        h = self.encoder(x)
        y = self.decoder(h)
        return y


In [7]:
class SelfExpression(nn.Module):
    def __init__(self, n):  
        super(SelfExpression, self).__init__()
        self.Coefficient = nn.Parameter(1.0e-8 * torch.ones(n, n, dtype=torch.float32), requires_grad=True)

    def forward(self, x):  
        y = torch.matmul(self.Coefficient, x)
        return y


# 谱聚类模块(module 3)

In [8]:
def thrC(C,ro): 
    C = C.detach().cpu()
    C = C.numpy()
    if ro < 1: 
        N = C.shape[1] 
        Cp = np.zeros((N,N))
        S = np.abs(np.sort(-np.abs(C),axis=0)) 
        Ind = np.argsort(-np.abs(C),axis=0) 
        for i in range(N): 
            cL1 = np.sum(S[:,i]).astype(float) 
            stop = False
            csum = 0
            t = 0
            while(stop == False): 
                csum = csum + S[t,i]
                if csum > ro*cL1:  
                    stop = True 
                    Cp[Ind[0:t+1,i],i] = C[Ind[0:t+1,i],i] 
                t = t + 1
    else:
        Cp = C

    return Cp


In [9]:
def post_proC(C, K, d, alpha):  
    C = 0.5*(C + C.T)
    r = d*K + 1
    U, S, _ = svds(C,r,v0 = np.ones(C.shape[0]))
    U = U[:,::-1] 
    S = np.sqrt(S[::-1])
    S = np.diag(S)    
    U = U.dot(S)
    U = normalize(U, norm='l2', axis = 1)       
    Z = U.dot(U.T)
    Z = Z * (Z>0) 
    L = np.abs(Z ** alpha) 
    L = L/L.max()   
    L = 0.5 * (L + L.T)    
    spectral = cluster.SpectralClustering(n_clusters=K, eigen_solver='arpack', affinity='precomputed',assign_labels='discretize')
    spectral.fit(L)
    grp = spectral.fit_predict(L)
    
    uu,ss,vv = svds(L, k = K) 
    
    return C, grp, uu


In [10]:
def form_structure_matrix(grp, num_classes):
    grp = torch.squeeze(grp) 
    Q = torch.zeros((grp.numel(), num_classes))
    for i in range(grp.numel()):
        j = grp[i]
        Q[i,j] = 1
    
    return Q

In [11]:
def form_Theta(Q):
    Theta = torch.zeros((Q.shape[0], Q.shape[0]))
    for i in range(Q.shape[0]):
        Qq = Q[i].repeat([Q.shape[0],1]) 
        Theta[i, :] = 1/2*torch.sum(torch.pow((Q - Qq),2), 1) 

    return Theta

# 加入自监督全连接层模块(module 2)

In [12]:
class RSDSC(nn.Module):
    def __init__(self, channels, kernels, num_sample, num_classes, num_fea):
        super(RSDSC, self).__init__()
        self.n = num_sample
        self.ae = ConvAE(channels, kernels)
        self.self_expression = SelfExpression(self.n)
        self.num_classes = num_classes
        self.num_fea = num_fea
        self.fc = nn.Sequential(torch.nn.Linear(self.num_fea, int(1/2*self.n)), torch.nn.Linear(int(1/2*self.n), self.num_classes))
        self.centers = nn.Parameter(torch.zeros(self.num_classes, self.num_classes)) 

    def forward(self, x):  
        z = self.ae.encoder(x)
        shape = z.shape 
        
        num_features = shape[1:4].numel() 
        flat_z = torch.reshape(z, (-1, num_features)) 
        fc_out = self.fc(flat_z) 
        
        distmat = torch.pow(fc_out, 2).sum(dim=1, keepdim=True).expand(self.n, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, self.n).t()
        distmat.addmm_(1, -2, fc_out, self.centers.t())     
               
        z = z.view(self.n, -1)  
        z_recon = self.self_expression(z)  
        
        z_recon_reshape = z_recon.view(shape) 
        x_recon = self.ae.decoder(z_recon_reshape)  
        
        return z, z_recon, x_recon, fc_out, distmat
    
    def loss_list(self, fc_out, distmat, grp, gamma3, gamma4, ratio):
        QQ = form_structure_matrix(grp, self.num_classes)
        Theta = form_Theta(QQ)
        Theta = Theta.cuda()
        Cq_loss_list = torch.sum(torch.pow(torch.abs(torch.t(self.self_expression.Coefficient)*Theta),1.0),dim=1)
        
        cross_entropy_list = torch.zeros(self.n).to(device)
        for i in range(self.n):
            tag = grp[i].view(-1)
            out = fc_out[i,:].view(-1, self.num_classes)
            cross_entropy_list[i] += F.cross_entropy(out, tag)
        
        classes = torch.arange(self.num_classes).long()
        classes = classes.cuda()
        grp = grp.unsqueeze(1).expand(self.n, self.num_classes)
        mask = grp.eq(classes.expand(self.n, self.num_classes)) 
        dist = distmat * mask.float()
        center_loss_list = dist.clamp(min=1e-12, max=1e+12).sum(dim=1) / self.n

        sp_loss_list = gamma3*Cq_loss_list + gamma4*(cross_entropy_list + ratio*center_loss_list)
        
        return sp_loss_list

    def loss_fn(self, x, z, z_recon, x_recon, sp_loss_list, gamma1, gamma2, V):
        loss_ae = F.mse_loss(x_recon, x, reduction='sum')  
        loss_selfExp = F.mse_loss(z_recon, z, reduction='sum')
        sp_loss = torch.sum(torch.mul(V, sp_loss_list))
        loss_coef = torch.sum(torch.pow(self.self_expression.Coefficient, 2)) 
        Coef_diag = torch.diag(self.self_expression.Coefficient)
        diag_list = torch.pow(Coef_diag, 2.0)
        diag_loss = torch.sum(diag_list)
        
        loss = sp_loss + gamma1 * loss_coef + diag_loss + loss_ae + gamma2*loss_selfExp

        return loss

In [13]:
rsdsc = RSDSC(channels, kernels, num_sample, num_classes, num_fea)
rsdsc.to(device)

RSDSC(
  (ae): ConvAE(
    (encoder): Sequential(
      (pad1): Conv2dSamePad()
      (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(2, 2))
      (relu1): ReLU(inplace)
      (pad2): Conv2dSamePad()
      (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2))
      (relu2): ReLU(inplace)
      (pad3): Conv2dSamePad()
      (conv3): Conv2d(3, 5, kernel_size=(3, 3), stride=(2, 2))
      (relu3): ReLU(inplace)
    )
    (decoder): Sequential(
      (deconv1): ConvTranspose2d(5, 3, kernel_size=(3, 3), stride=(2, 2))
      (padd0): ConvTranspose2dSamePad()
      (relud0): ReLU(inplace)
      (deconv2): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(2, 2))
      (padd1): ConvTranspose2dSamePad()
      (relud1): ReLU(inplace)
      (deconv3): ConvTranspose2d(3, 1, kernel_size=(3, 3), stride=(2, 2))
      (padd2): ConvTranspose2dSamePad()
      (relud2): ReLU(inplace)
    )
  )
  (self_expression): SelfExpression()
  (fc): Sequential(
    (0): Linear(in_features=80, out_features=200, b

# train(读取预训练模型，跳过stage1)

In [14]:
#Diffusion Processing

#Sparse the affinity matrix by k-nearst neighbor
def knnSparse(W,k):
    n = W.shape[0]
    idx_knn = np.argsort(-W, 1) 
    W_knn = np.zeros((n,n))
    for i in range(n):
        W_knn[i, idx_knn[i, 0:k]] = W[i, idx_knn[i, 0:k]]
    W_knn = (W_knn + W_knn.T)/2
    return W_knn

def IterativeDiffusionTPGKNN(W, k, shrink):
    W = 1/2*(np.abs(W) + np.abs(W.T))   
    #Pre-processing of affinity matrix W
    d = np.sum(W,1)
    D = np.diag(d + np.spacing(1))
    W = W - np.diag(np.diag(W)) + D

    #Normalization:W = W ./ repmat(sum(W, 2)+eps, 1, n)
    d = np.sum(W,1)
    D = np.diag(d + np.spacing(1))
    W = np.dot(inv(D), W) #nomalize the row of W
    S = shrink*knnSparse(W, k)
    WW = S
    maxIter = 50 
    epsilon = 1e-2
     
    for t in range(maxIter):
        temp = np.dot((np.dot(S,WW)),S.T) + np.eye(max(WW.shape))  
        if np.linalg.norm(temp-WW, ord='fro') < epsilon: 
            break
        WW = temp
    
    WW = knnSparse(WW, k)
    
    return WW

In [15]:
#Sample weight estimating

def R_graph(R, p, rw_time):
    #compute transition P from R 
    R_norm = np.linalg.norm(np.abs(R), ord=p, axis=1, keepdims=True)
    P = R/(R_norm + np.spacing(1)) #transition matrix
    #compute \pi from P 
    N = R.shape[0]
    PI = 1/N*np.ones((1,N)) #state probability distribution
    if rw_time == 0:
        PI_bar = np.ones((1,N))
    else:
        PI_bar = np.zeros((1,N))
        for i in range(rw_time):
            PI = np.dot(PI, P)
            PI_bar = PI_bar + PI  
        PI_bar = PI_bar/rw_time
        
    return PI_bar

def cluster_rg(C, grp, p, rw_time, x, epoch):
    if isinstance(C, torch.Tensor):
        C = C.detach().cpu().numpy()
    if isinstance(grp, torch.Tensor):
        grp = grp.detach().cpu().numpy()
    grp = np.squeeze(grp)
    grp_classes = np.unique(grp)
    num_samples = grp.shape[0]
    V = np.zeros(num_samples)
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.squeeze(x).transpose(0,2,1)
    for i in grp_classes.tolist():
        location = np.where((i==grp)==True)
        if location[0].shape[0] == 1:
            v = np.array([[0]])
        else:
            x_grp = x[location][:][:]
            R = np.squeeze(C[location][:,location])
            v = R_graph(R, p, rw_time)
        for j,k in zip(np.nditer(location), range(location[0].shape[0])):
            V[j] += v[:,k]
            
    V = V.reshape(1,-1)
    # normalization
    _range = np.max(V,1) - np.min(V,1)
    if _range != 0:
        V = (V - np.min(V,1)) / _range * 0.5 + 0.5
    return V 

In [16]:
def train(model, x, y, lr, device, alpha, dim_subspace, ro, show, t0, t_dp, k_dp, gamma1, gamma2, gamma3, gamma4, ratio, update, p, rw_time, shrink):
    
    optimizer = optim.Adam([{"params" : model.fc.parameters()},
                            {"params" : model.ae.parameters()},
                            {"params" : model.self_expression.parameters()},
                            {"params" : model.centers, lr : 0.5}],
                           lr=lr)
    
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32, device=device)
    x = x.to(device)
    if isinstance(y, torch.Tensor):
        y = y.to('cpu').numpy()

    K = len(np.unique(y)) 
    
    C = model.self_expression.Coefficient
    C0 = thrC(C, alpha)
    _, grp1, _ = post_proC(C0, K, dim_subspace, ro)
    
    grp1 = torch.tensor(grp1).to(device)

    x = Variable(x, requires_grad=False)
    
    best_performence = 0
       
    update_counter = 0
    
    coa, cob = 0.5, 0.5
    
    for epoch in range(epochs):
        if epoch <= t0: 
            z, z_recon, x_recon, fc_out, distmat = model(x)
            sp_loss_list = model.loss_list(fc_out, distmat, grp1, gamma3, gamma4, ratio)
    
            if epoch > t0:
                C = model.self_expression.Coefficient
                C1 = thrC(C, alpha)
                V1 = coa*cluster_rg(C1, grp1, p, rw_time, x, epoch) 
                V2 = cob*R_graph(C1, p, rw_time)
                V = V1 + V2
                V = torch.from_numpy(V).float().cuda()
            else:
                V1 = coa*cluster_rg(C0, grp1, p, rw_time, x, epoch) 
                V2 = cob*R_graph(C0, p, rw_time) 
                V = V1 + V2
                V = torch.from_numpy(V).float().cuda()
                
                
            loss = model.loss_fn(x, z, z_recon, x_recon, sp_loss_list, gamma1, gamma2, V)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('Epoch %02d:loss=%.4f' %(epoch, loss.item() / grp1.shape[0]))
            
            
        else:
            C = model.self_expression.Coefficient
            C2 = thrC(C, alpha)
            
            if epoch <= t_dp: 
                print('stage2')
                _, grp2, _ = post_proC(C2, K, dim_subspace, ro)
                  
                if (update_counter % update == 0) and (update_counter != 0): 
                    V1 = coa*cluster_rg(C2, grp2, p, rw_time, x, epoch) 
                    V2 = cob*R_graph(C2, p, rw_time)
                    V = V1 + V2
                    V = torch.from_numpy(V).float().cuda()
                    update_counter = 1
                elif update_counter == 0:
                    V1 = coa*cluster_rg(C2, grp2, p, rw_time, x, epoch)
                    V2 = cob*R_graph(C2, p, rw_time)
                    V0 = V1 + V2
                    V0 = torch.from_numpy(V0).float().cuda()
                    V = V0
                    update_counter += 1
                else:
                    update_counter += 1
                    
            else:  
                print('stage3')
                C3 = IterativeDiffusionTPGKNN(C2, k_dp, shrink)
                _, grp2, _ = post_proC(C3, K, dim_subspace, ro)
                
                if (update_counter % update == 0) and (update_counter != 0):
                    V1 = coa*cluster_rg(C3, grp2, p, rw_time, x, epoch) 
                    V2 = cob*R_graph(C3, p, rw_time)
                    V = V1 + V2
                    V = torch.from_numpy(V).float().cuda()
                    update_counter = 1
                elif update_counter == 0:
                    V1 = coa*cluster_rg(C3, grp2, p, rw_time, x, epoch)
                    V2 = cob*R_graph(C3, p, rw_time)
                    V0 = V1 + V2 
                    V0 = torch.from_numpy(V0).float().cuda()
                    V = V0
                    update_counter += 1
                else:
                    update_counter += 1

            print('Epoch %02d: acc=%.4f, nmi=%.4f' %(epoch, acc(y, grp2), nmi(y, grp2)))
            if acc(y, grp2) > best_performence:
                best_performence = acc(y, grp2)
            
            grp2 = torch.tensor(grp2).to(device)
            z, z_recon, x_recon, fc_out, distmat = model(x)
            sp_loss_list = model.loss_list(fc_out, distmat, grp2, gamma3, gamma4, ratio)
            loss = model.loss_fn(x, z, z_recon, x_recon, sp_loss_list, gamma1, gamma2, V)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    print('best_performence = %.4f'%best_performence)

In [17]:
show_freq=1

In [18]:
# load the pretrained weights 
ae_state_dict = torch.load('pretrained_weights_original/%s.pkl' % 'orl')
rsdsc.ae.load_state_dict(ae_state_dict) 
print("Pretrained ae weights are loaded successfully.")

train(rsdsc, x, y, lr, device=device, alpha=alpha, dim_subspace=dim_subspace, ro=ro, show=show_freq,\
      t0=t0, t_dp=t_dp, k_dp=k_dp, gamma1=gamma1, gamma2=gamma2, gamma3=gamma3, gamma4=gamma4,\
      ratio=ratio, update=update, p=p, rw_time=rw_time, shrink=shrink)

Pretrained ae weights are loaded successfully.
Epoch 00:loss=15238909.4400
Epoch 01:loss=15329844.4800
Epoch 02:loss=15370977.2800
Epoch 03:loss=15395361.2800
Epoch 04:loss=15407050.2400
Epoch 05:loss=15409199.3600
stage2
Epoch 06: acc=0.3125, nmi=0.5449
stage2
Epoch 07: acc=0.2750, nmi=0.5348
stage2
Epoch 08: acc=0.3225, nmi=0.5496
stage2
Epoch 09: acc=0.3275, nmi=0.5622
stage2
Epoch 10: acc=0.3075, nmi=0.5459
stage2
Epoch 11: acc=0.3275, nmi=0.5580
stage2
Epoch 12: acc=0.3050, nmi=0.5420
stage3
Epoch 13: acc=0.1725, nmi=0.3952
stage3
Epoch 14: acc=0.1700, nmi=0.3904
stage3
Epoch 15: acc=0.1900, nmi=0.4031
stage3
Epoch 16: acc=0.1800, nmi=0.3884
stage3
Epoch 17: acc=0.1750, nmi=0.3908
stage3
Epoch 18: acc=0.1775, nmi=0.4014
stage3
Epoch 19: acc=0.1800, nmi=0.4086
stage3
Epoch 20: acc=0.1775, nmi=0.4035
stage3
Epoch 21: acc=0.1750, nmi=0.4018
stage3
Epoch 22: acc=0.1750, nmi=0.3963
stage3
Epoch 23: acc=0.1750, nmi=0.3934
stage3
Epoch 24: acc=0.1900, nmi=0.4063
stage3
Epoch 25: acc=0.19

stage3
Epoch 203: acc=0.6350, nmi=0.7934
stage3
Epoch 204: acc=0.6300, nmi=0.7981
stage3
Epoch 205: acc=0.6375, nmi=0.7999
stage3
Epoch 206: acc=0.6400, nmi=0.8004
stage3
Epoch 207: acc=0.6525, nmi=0.8109
stage3
Epoch 208: acc=0.6350, nmi=0.8075
stage3
Epoch 209: acc=0.6475, nmi=0.8173
stage3
Epoch 210: acc=0.6525, nmi=0.8131
stage3
Epoch 211: acc=0.6525, nmi=0.8060
stage3
Epoch 212: acc=0.6475, nmi=0.8104
stage3
Epoch 213: acc=0.6400, nmi=0.7988
stage3
Epoch 214: acc=0.6325, nmi=0.7992
stage3
Epoch 215: acc=0.6500, nmi=0.7997
stage3
Epoch 216: acc=0.6600, nmi=0.8156
stage3
Epoch 217: acc=0.6600, nmi=0.8170
stage3
Epoch 218: acc=0.6475, nmi=0.8142
stage3
Epoch 219: acc=0.6500, nmi=0.8114
stage3
Epoch 220: acc=0.6300, nmi=0.7944
stage3
Epoch 221: acc=0.6250, nmi=0.7964
stage3
Epoch 222: acc=0.6525, nmi=0.8152
stage3
Epoch 223: acc=0.6375, nmi=0.8095
stage3
Epoch 224: acc=0.6500, nmi=0.8098
stage3
Epoch 225: acc=0.6375, nmi=0.8076
stage3
Epoch 226: acc=0.6475, nmi=0.8157
stage3
Epoch 227

stage3
Epoch 403: acc=0.8150, nmi=0.8971
stage3
Epoch 404: acc=0.8125, nmi=0.8956
stage3
Epoch 405: acc=0.8300, nmi=0.9077
stage3
Epoch 406: acc=0.8275, nmi=0.9048
stage3
Epoch 407: acc=0.8350, nmi=0.9107
stage3
Epoch 408: acc=0.8375, nmi=0.9092
stage3
Epoch 409: acc=0.8300, nmi=0.9089
stage3
Epoch 410: acc=0.8375, nmi=0.9103
stage3
Epoch 411: acc=0.8475, nmi=0.9107
stage3
Epoch 412: acc=0.8275, nmi=0.9128
stage3
Epoch 413: acc=0.8200, nmi=0.9054
stage3
Epoch 414: acc=0.8275, nmi=0.9087
stage3
Epoch 415: acc=0.8125, nmi=0.8988
stage3
Epoch 416: acc=0.8225, nmi=0.9054
stage3
Epoch 417: acc=0.8325, nmi=0.9083
stage3
Epoch 418: acc=0.8375, nmi=0.9164
stage3
Epoch 419: acc=0.8275, nmi=0.9039
stage3
Epoch 420: acc=0.8375, nmi=0.9117
stage3
Epoch 421: acc=0.8325, nmi=0.9083
stage3
Epoch 422: acc=0.8400, nmi=0.9125
stage3
Epoch 423: acc=0.8150, nmi=0.8972
stage3
Epoch 424: acc=0.8200, nmi=0.9001
stage3
Epoch 425: acc=0.8200, nmi=0.9001
stage3
Epoch 426: acc=0.8250, nmi=0.9059
stage3
Epoch 427

stage3
Epoch 603: acc=0.8450, nmi=0.9221
stage3
Epoch 604: acc=0.8425, nmi=0.9241
stage3
Epoch 605: acc=0.8425, nmi=0.9211
stage3
Epoch 606: acc=0.8125, nmi=0.9134
stage3
Epoch 607: acc=0.8075, nmi=0.9095
stage3
Epoch 608: acc=0.8300, nmi=0.9208
stage3
Epoch 609: acc=0.8300, nmi=0.9194
stage3
Epoch 610: acc=0.8275, nmi=0.9161
stage3
Epoch 611: acc=0.8075, nmi=0.9103
stage3
Epoch 612: acc=0.8100, nmi=0.9118
stage3
Epoch 613: acc=0.8325, nmi=0.9209
stage3
Epoch 614: acc=0.8175, nmi=0.9119
stage3
Epoch 615: acc=0.8275, nmi=0.9152
stage3
Epoch 616: acc=0.8275, nmi=0.9155
stage3
Epoch 617: acc=0.8275, nmi=0.9164
stage3
Epoch 618: acc=0.8125, nmi=0.9131
stage3
Epoch 619: acc=0.8275, nmi=0.9192
stage3
Epoch 620: acc=0.8150, nmi=0.9135
stage3
Epoch 621: acc=0.8250, nmi=0.9174
stage3
Epoch 622: acc=0.8300, nmi=0.9185
stage3
Epoch 623: acc=0.8225, nmi=0.9119
stage3
Epoch 624: acc=0.8300, nmi=0.9167
stage3
Epoch 625: acc=0.8325, nmi=0.9130
stage3
Epoch 626: acc=0.8350, nmi=0.9166
stage3
Epoch 627

stage3
Epoch 803: acc=0.8650, nmi=0.9221
stage3
Epoch 804: acc=0.8525, nmi=0.9173
stage3
Epoch 805: acc=0.8525, nmi=0.9159
stage3
Epoch 806: acc=0.8675, nmi=0.9278
stage3
Epoch 807: acc=0.8750, nmi=0.9286
stage3
Epoch 808: acc=0.8700, nmi=0.9278
stage3
Epoch 809: acc=0.8500, nmi=0.9197
stage3
Epoch 810: acc=0.8800, nmi=0.9268
stage3
Epoch 811: acc=0.8675, nmi=0.9308
stage3
Epoch 812: acc=0.8450, nmi=0.9191
stage3
Epoch 813: acc=0.8550, nmi=0.9163
stage3
Epoch 814: acc=0.8525, nmi=0.9229
stage3
Epoch 815: acc=0.8425, nmi=0.9248
stage3
Epoch 816: acc=0.8650, nmi=0.9342
stage3
Epoch 817: acc=0.8575, nmi=0.9282
stage3
Epoch 818: acc=0.8600, nmi=0.9317
stage3
Epoch 819: acc=0.8625, nmi=0.9319
stage3
Epoch 820: acc=0.8600, nmi=0.9304
stage3
Epoch 821: acc=0.8500, nmi=0.9238
stage3
Epoch 822: acc=0.8500, nmi=0.9262
stage3
Epoch 823: acc=0.8550, nmi=0.9264
stage3
Epoch 824: acc=0.8575, nmi=0.9295
stage3
Epoch 825: acc=0.8500, nmi=0.9240
stage3
Epoch 826: acc=0.8500, nmi=0.9231
stage3
Epoch 827